<a href="https://colab.research.google.com/github/floriandendorfer/demand-estimation/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load packages.

In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [11]:
!wget -O data.csv https://raw.githubusercontent.com/floriandendorfer/demand-estimation/main/data.csv?token=GHSAT0AAAAAACRKP3ZXRJSBTHSJ2NGVIUTAZTAZFTA

--2024-06-05 20:23:13--  https://raw.githubusercontent.com/floriandendorfer/demand-estimation/main/data.csv?token=GHSAT0AAAAAACRKP3ZXRJSBTHSJ2NGVIUTAZTAZFTA
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22876 (22K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  22.34K  --.-KB/s    in 0.002s  

2024-06-05 20:23:13 (13.8 MB/s) - ‘data.csv’ saved [22876/22876]



Load data.

In [12]:
data = pd.read_csv('data.csv',index_col=0)

In [13]:
print(data.columns)

Index(['county', 'Häagen-Dazs', 'price', 'fuel cost', 'sales', 'county size',
       'top 10% income'],
      dtype='object')


Define the market share of each firm (Ben & Jerry's, Hägen-Dazs) in each county based on the number of ice cream units sold.

In [14]:
data['s'] = data['sales']/data['county size']


Define the 'outside good' market share for each county (here the 'outside good' is any ice cream sold other than Ben & Jerry's or Hägen-Dazs).

In [15]:
data['s0'] = 1 - data.groupby(['county'])['s'].transform('sum')

Transform the market shares to back out the **mean utilities**. The transformed market share is going to be the **dependent** variable in the OLS regression we run next.

In [16]:
Y = np.log(data['s']) - np.log(data['s0'])

In the OLS regression, the **independent** variables are going to be the price and the Häagen-Dazs dummy variable, plus a constant.

In [17]:
X=sm.add_constant(data[['price','Häagen-Dazs']])

In [18]:
ols = sm.OLS(Y,X)
ols_result = ols.fit(cov_type='HC3')
ols_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.596
Method:                 Least Squares   F-statistic:                     296.3
Date:                Wed, 05 Jun 2024   Prob (F-statistic):           1.84e-79
Time:                        20:23:25   Log-Likelihood:                -228.34
No. Observations:                 400   AIC:                             462.7
Df Residuals:                     397   BIC:                             474.7
Df Model:                           2                                         
Covariance Type:                  HC3                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.6042      0.143    -11.190      0.000      -1.885      -1.323
price          -0.1275      0.057     -2.256      0.024      -0.238      -0.017
Häagen-Dazs    -1.0612      0.044    -24.270      0.000      -1.147      -0.975
==============================================================================
Omnibus:                       10.208   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.006   Jarque-Bera (JB):                9.458
Skew:                          -0.323   Prob(JB):                      0.00884
Kurtosis:                       2.612   Cond. No.                         20.0
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""